In [ ]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 6.1 MB/s eta 0:00:00m eta 0:00:010:01:01


In [2]:
import gurobipy as gp
from gurobipy import GRB, quicksum

In [3]:
n_real = 3
n_virtual = 2
n_total = n_real + n_virtual  # 9
k = 2

# Priorities of each equipment (p_j) - real + virtual
# p_j = [10, 2, 4, 5, 6, 15, 20, 0, 0]  # len=9, últimos 2 virtuais com prioridade 0
p_j = [0, 2, 4, 10, 0]
# Time to inspect equipment (t_j) - real + virtual
# t_j = [30, 45, 20, 35, 50, 25, 60, 0, 0]  # len=9, últimos 2 virtuais 0
t_j = [0, 45, 20, 30, 0]
# S_ij matrix 9x9 (real+virtual)
# S_ij = [
#     [0,   0,  0,  0,  0,  0,  0, 0, 0],  # virtual node 1
#     [0,12,  0, 13, 15, 17, 14, 20, 0],
#     [0,10, 13,  0, 12,  8, 19, 18, 0],
#     [0,18, 15, 12,  0, 16, 21, 11, 0],
#     [0,20, 17,  8, 16,  0, 13, 19, 0],
#     [0,9,  14, 19, 21, 13,  0, 10, 0],
#     [0,22, 20, 18, 11, 19, 10,  0, 0],
#     [0,  12, 10, 18, 20,  9, 22, 0, 0],
#     [0,0,   0,  0,  0,  0,  0,  0, 0],  # virtual node 2
# ]

S_ij = [
    [0, 00, 00, 00, 0],  # virtual node 1
    [0, 00, 10, 20, 0],
    [0, 10, 00, 30, 0],
    [0, 20, 30, 00, 0],
    [0, 00, 00, 00, 0],  # virtual node 2
]

# Initial shift for each team to each real equipment (size 7)
# d_mj = [
#     [10, 15, 12, 20, 25, 18, 16],  # team 0
#     [11, 13, 14, 18, 20, 19, 21],  # team 1
# ]

d_mj = [
    [0, 10, 15, 100, 0],  # team 0
    [0, 11, 13, 110, 0],  # team 1
]


# q_jl - calculate for real equipments only
q_jl = [[round(p / (l + 1), 2) for l in range(n_real+1)] for p in p_j]

In [4]:
for i in range(2, n_real+1):
    print(i)
print(q_jl)
print(q_jl[3][1])

2
3
[[0.0, 0.0, 0.0, 0.0], [2.0, 1.0, 0.67, 0.5], [4.0, 2.0, 1.33, 1.0], [10.0, 5.0, 3.33, 2.5], [0.0, 0.0, 0.0, 0.0]]
5.0


In [110]:
m = gp.Model()

# Create variables
C_j = m.addVars(n_total, name='C_j', vtype=gp.GRB.CONTINUOUS, lb=0)
Y_ilm = m.addVars(n_total, n_total, k, name='Y_ilm', vtype=gp.GRB.BINARY)
X_ijm = m.addVars(n_total, n_total, k, name='X_ijm', vtype=gp.GRB.BINARY)
C_max = m.addVar(name='C_max', vtype=gp.GRB.CONTINUOUS, lb=0)

#Create objective function

# Z_p
Z_p = quicksum(q_jl[i][l] * Y_ilm[i, l, m]
               for i in range(n_real+1)
               for l in range(n_real+1)
               for m in range(k))

# TFT
# Parte 1: sequência das inspeções (i precede j)
flow_time = quicksum(
    (S_ij[i][j] + t_j[j]) * X_ijm[i, j, m]
    for i in range(n_real + 1)
    for j in range(1, n_real + 2)
    if i != j
    for m in range(k)
)

# Parte 2: deslocamento inicial da equipe m até o primeiro equipamento j
initial_travel = quicksum(
    d_mj[m][j] * Y_ilm[j, 1, m]
    for j in range(1, n_real + 2)
    for m in range(k)
)

TFT = flow_time + initial_travel

# Z_t
Z_t = C_max + TFT

# Final objective function
m.setObjective(10 * Z_p - Z_t, GRB.MAXIMIZE)
# m.setObjective(Z_t, GRB.MINIMIZE)
# m.setObjective(Z_p, GRB.MAXIMIZE)

# -----------------------------
# Create constraints
# -----------------------------

# # S1 -Makespan Linear
for j in range(1, n_real + 1):
    m.addConstr(C_max >= C_j[j], name="S1 j=%d" % j)

# S2 - All equipment inspections are just a predecessor
for j in range(1, n_real + 1):
    m.addConstr(
        gp.quicksum(X_ijm[i, j, m_id] for i in range(n_real + 1) for m_id in range(k) if i != j ) == 1,
        name="S2 j=%d" % j
    )

# S3 – Each team must finish its inspection at node n+1 (node 11)
for m_id in range(k):
    m.addConstr(
        gp.quicksum(X_ijm[i, n_real + 1, m_id] for i in range(1, n_real + 1)) == 1,
        name="S3 m=%d" % m_id
    )

#S4 – Each team starts inspection from virtual node 0
for m_id in range(k):
    m.addConstr(
        gp.quicksum(X_ijm[0, j, m_id] for j in range(1, n_real + 1)) == 1,
        name="S4 m=%d" % m_id
    )

# # S4.2 – Each team starts at node 0 in position 0
for m_id in range(k):
    m.addConstr(Y_ilm[0, 0, m_id] == 1, name="S4.2 m=%d" % m_id)

# S5 – All equipment inspections have only one successor
for i in range(1, n_real + 1):
    m.addConstr(
        gp.quicksum(X_ijm[i, j, m_id] for j in range(n_real + 2) if j != i for m_id in range(k)) == 1,
        name="S5 i=%d" % i
    )

#S7 – Temporal constraint: Cj ≥ Ci + Sij + tj if Xijm = 1 (big-M formulation)
M = 1e4  # Big M constant
for i in range(n_real + 1):  # inclui o nó virtual 0
    for j in range(1, n_real + 2):  # de 1 até n+1
        if i == j:
            continue
        for m_id in range(k):
            if i == 0:
                # caso inicial: deslocamento inicial + tempo de inspeção
                m.addConstr(
                    C_j[j] >= -M + (t_j[j] + d_mj[m_id][j] + M) * X_ijm[i, j, m_id],
                    name=f"S7 i=0 j={j} m={m_id}"
                )
            else:
                # caso geral: tempo da tarefa anterior + setup + tempo de inspeção
                m.addConstr(
                    C_j[j] >= C_j[i] - M + (t_j[j] + S_ij[i][j] + M) * X_ijm[i, j, m_id],
                    name=f"S7 i={i} j={j} m={m_id}"
                )

# S8 – Each equipment is inspected exactly once by one team and at one position
for i in range(1, n_real + 1):
    m.addConstr(
        gp.quicksum(Y_ilm[i, l, m_id] for l in range(1, n_real+1) for m_id in range(k)) == 1,
        name="S8 i=%d" % i
    )

# # S9 – Inspection positions must be consecutive (no gaps in assignment)
for m_id in range(k):
    for l in range(2, n_real+1):
        m.addConstr(
            gp.quicksum(Y_ilm[i, l - 1, m_id] for i in range(n_real + 1)) >=
            gp.quicksum(Y_ilm[i, l, m_id] for i in range(n_real + 1)),
            name="S9 l=%d m=%d" % (l, m_id)
        )

#S10 – Inspection sequencing of X_ijm, positions must be consecutive
# for i in range(1, n_real + 1): 
#     for m_id in range(k):  
#         m.addConstr(
#             gp.quicksum(X_ijm[i, j, m_id] for j in range(1, n_real + 2)) +
#             gp.quicksum(X_ijm[j, i, x_id]
#                         for j in range(n_real + 1)
#                         for x_id in range(k) if x_id != m_id)
#             <= 1,
#             name=f"S10_i={i}_m={m_id}"
#         )

#S11 - Same position and same team must inspect only one equipment
for m_id in range(k):
    for l in range(0, n_real+1):
        m.addConstr(
            gp.quicksum(Y_ilm[i, l, m_id] for i in range(n_real + 1)) <= 1,
            name=f"S11_l={l}_m={m_id}"
        )
# #S12 - The equipament zero must be inspected only once by each team
for m_id in range(k):
    m.addConstr(
        gp.quicksum(Y_ilm[0, l, m_id] for l in range(0, n_real)) == 1,
        name="S12_m={m_id}"
    )
#S13 - 
for m_id in range(k):  
    for l in range(1, n_real + 1):
        for i in range(0, n_real + 1):
            for j in range(1, n_real + 1):
                if i == j:
                    continue
                m.addConstr(
                    Y_ilm[i, l - 1, m_id] + Y_ilm[j, l, m_id] - X_ijm[i, j, m_id] <= 1,
                    name=f"S13_i={i}_j={j}_l={l}_m={m_id}"
                )
# for m_id in range(k):  
#     for i in range(0, n_real + 1):
#         for j in range(1, n_real + 2):
#             if i == j:
#                 continue
#             m.addConstr(
#                 gp.quicksum(Y_ilm[i, l - 1, m_id] + Y_ilm[j, l, m_id] for l in range(1, n_real + 1)) - X_ijm[i, j, m_id] <= 1
#             )

#S14 -
# for j in range(1, n_real + 1):
#     for m_id in range(k):
#         m.addConstr(
#             gp.quicksum(X_ijm[i, j, m_id] for i in range(n_real + 1) if i != j) <= gp.quicksum(Y_ilm[j, l, m_id] for l in range(1, n_real + 1)),
#             name=f"S14_j={j}_m={m_id}"
#         )

# #S15 - 
for j in range(1, n_real + 1):
    for m_id in range(k):
        m.addConstr(
            gp.quicksum(X_ijm[i, j, m_id] for i in range(n_real + 1) if i != j) >= gp.quicksum(X_ijm[j, x, m_id] for x in range(1, n_real + 2) if x != j),
            name=f"S15_j={j}_m={m_id}"
        )

In [111]:
m.optimize()

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 24.04.2 LTS")

CPU model: AMD FX(tm)-6300 Six-Core Processor, instruction set [SSE2|AVX]
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads

Optimize a model with 118 rows, 106 columns and 418 nonzeros
Model fingerprint: 0xa6617fa7
Variable types: 6 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Found heuristic solution: objective -230.7000000
Presolve removed 42 rows and 60 columns
Presolve time: 0.01s
Presolved: 76 rows, 46 columns, 312 nonzeros
Variable types: 4 continuous, 42 integer (42 binary)

Root relaxation: objective -1.733167e+02, 22 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     

In [114]:
if m.status == gp.GRB.OPTIMAL:
    print("Optimal solution found!")

    # Print objective value
    print("Objective value:", m.objVal)

    # Print decision variable values that are non-zero
    for v in m.getVars():
        if v.x != 0:
            print(v.varName, "=", v.x)

else:
    print("Optimal solution not found.")

Optimal solution found!
Objective value: -177.7
C_j[1] = 55.0
C_j[2] = 33.0
C_j[3] = 93.0
C_j[4] = 93.0
Y_ilm[0,0,0] = 1.0
Y_ilm[0,0,1] = 1.0
Y_ilm[1,1,0] = 1.0
Y_ilm[2,1,1] = 1.0
Y_ilm[3,2,1] = 1.0
X_ijm[0,1,0] = 1.0
X_ijm[0,2,1] = 1.0
X_ijm[1,4,0] = 1.0
X_ijm[2,3,1] = 1.0
X_ijm[3,4,1] = 1.0
C_max = 93.0


[[0.0, 0.0, 0.0, 0.0], [2.0, 1.0, 0.67, 0.5], [4.0, 2.0, 1.33, 1.0], [10.0, 5.0, 3.33, 2.5], [0.0, 0.0, 0.0, 0.0]]